<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/Geoapify%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%93%B1%EC%B9%98%EC%84%A0%EB%8F%84_%EA%B7%B8%EB%A6%AC%EA%B8%B0_20210831.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Geoapify를 이용한 등치선도 그리기

##이름이나 주소값으로 위치값 알아내기(지오코딩, Geocoding)
여기서는 [Geoapify](https://www.geoapify.com/)의 무료 API 서비스를 이용해 지오코딩 및 몇가지 지오프로세싱(Geoprocessing)을 실시합니다.<br>
보다 세부적인 내용 또는 작동 원리는 API PLAYGROUND 와 DOCUMENTATION을 확인바랍니다.

### 사용된 코드(숨김)

In [1]:
import requests
import json
from requests.structures import CaseInsensitiveDict
import urllib
import folium
from folium.plugins import *
import pandas as pd
import numpy as np

In [2]:
#Geoapify API 서비스를 사용해 지오코딩 하기(느림)
def Geoapify_geocoding(search_name, report='y', apikey='d079ec0e5b1a4c3d9b4301d3618abfb4'):
  #사전 준비
  lats, lons = [], []
  for num, each in enumerate([search_name]):
    url = "https://api.geoapify.com/v1/geocode/search?text={}&apiKey={}".format(each, apikey)
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    resp = requests.get(url, headers=headers)
    statcode = resp.status_code
    resp_data = resp.json()
    
    if statcode == 200 and len(resp_data['features']) > 0:
      re_data = resp_data['features'][0]['properties']
      try:
        lats.append(re_data['lat'])
      except:
        lats.append(0.0)
      try:
        lons.append(re_data['lon'])
      except:
        lons.append(0.0)
    else:
      lats.append(0.0)
      lons.append(0.0)
      if report == 'y':
        print('"{}" 를 찾지 못했습니다'.format(each))

  return lats, lons

In [3]:
def spatial_visualizing(xcoord=None, ycoord=None, zoom= 7,
                        minimap=True, measure=True, position=True, fullscrean=True):
  m = folium.Map(location=[np.mean(ycoord), np.mean(xcoord)], zoom_start=zoom,
                 control_scale=True) #스케일바 추가
  #지도 타일 종류 추가
  for i in ['Stamen Terrain', 'Stamen Water Color', 'cartodbpositron']:
    folium.TileLayer(i).add_to(m)

  #미니맵 추가
  if minimap==True:
    m.add_child(MiniMap())

  #거리측량기 추가
  if measure==True:
    m.add_child(MeasureControl())

  #좌표기 추가
  if position==True:
    MousePosition(prefix='경위도 좌표:').add_to(m)

  #전체화면모드 추가
  if fullscrean==True:
    Fullscreen(title='전체화면', title_cancel='돌아가기', force_separate_button=True).add_to(m)

  return m

In [4]:
def Geoapify_Isoline(lat=None, lon=None, cal_type='distance', cal_mode='drive',
                     search_range=None, apiKey='01dfae275cd34bfcab046b255952bd49'):
  #혹시 모를 에러 방지를 위해 계산식을 사용하지 않는 모든 부분은 str로 한번 더 처리해준다.

  if type(search_range) == list:
    all_ranges = ['{}={}'.format('range', i) for i in search_range]
    ranges = '&'.join(all_ranges)
  else:
    ranges = '{}={}'.format('range', search_range)
  
  url = 'https://api.geoapify.com/v1/isoline?lat={}&lon={}&type={}&mode={}&{}&apiKey={}'.format(lat, lon, cal_type, cal_mode, ranges, apiKey)
  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"
  resp = requests.get(url, headers=headers)
  statcode = resp.status_code
  resp_data = resp.json()

  return statcode, resp_data

In [5]:
def df_and_bins_maker(resp_json=None, col_name='range'):
  id_list, range_list = [], []
  for i in range(len(resp_json['features'])):
    id_list.append(resp_json['features'][i]['properties']['id'])
    range_list.append(resp_json['features'][i]['properties'][col_name])
  id_range = [id_list, range_list]
  df = pd.DataFrame(id_range).transpose()
  df.columns = ['id',col_name]
  bins = list(df[col_name].quantile([0, 0.25, 0.5, 0.75, 1]))
  return df, bins

In [6]:
def Geoapify_Geometry(target_ids=None, operation='intersection', apiKey='ce8069542fd94ca2af8b17934f9510ce'):
  #"operation" must be one of [intersection, union]
  all_ids = ['{}={}'.format('id', i) for i in target_ids]
  ids = '&'.join(all_ids)
  url = 'https://api.geoapify.com/v1/geometry?operation={}&{}&&apiKey={}'.format(operation, ids, apiKey)
  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"
  resp = requests.get(url, headers=headers)
  statcode = resp.status_code
  resp_data =resp.json()
  return statcode, resp_data

###코드 실행부(Ctrl+F10)

####데이터 지오코딩 및 시각화하기

In [7]:
#@title 중심점의 주소나 이름을 입력해주세요
location = '\uC22D\uB840\uBB38' #@param {type:"string"}
#@markdown (cf. 한국교원대  or  충청북도 청주시 흥덕구 태성탑연로 250)
#@markdown ### API Key를 입력해주세요
api_key = 'd079ec0e5b1a4c3d9b4301d3618abfb4' #@param {type:"string"}
#@markdown (cf. d079ec0e5b1a4c3d9b4301d3618abfb4, Geoapify에서 발급)
ylat, xlon = Geoapify_geocoding(location, report='y', apikey=api_key)
map = spatial_visualizing(xcoord=xlon, ycoord=ylat, zoom= 15)
#중심점 추가
for num, latitude in enumerate(ylat):
  popup = folium.Popup('위도: {}<br>경도: {}'.format(round(xlon[num], 4), round(ylat[num], 4)), max_width=300)
  folium.Marker(location=[ylat[num], xlon[num]], popup=popup, tooltip=location,
                icon=folium.Icon(color='red', icon='info-sign')).add_to(map)
#레이어 컨트롤러 추가
folium.LayerControl().add_to(map)
map

In [8]:
#지도 저장하기
map.save('geocoding.html')
from google.colab import files
files.download('/content/geocoding.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##특정지점의 영향 범위 표시하기1(Ctrl+Enter)<br>(등치선도, Isoline Map)

In [9]:
#@title 중심점의 주소나 이름을 입력해주세요
location = '\uD55C\uAD6D\uAD50\uC6D0\uB300\uD559\uAD50' #@param {type:"string"}
#@markdown (cf. 한국교원대  or  충청북도 청주시 흥덕구 태성탑연로 250)
#@markdown ### 등치선도 범위를 입력해주세요
isolines = '2500' #@param {type:"string"}
#@markdown (cf.500 300 100  |  단, distance는 미터, time은 초단위로 입력해주세요)
#@markdown ### API Key를 입력해주세요
api_key = 'd079ec0e5b1a4c3d9b4301d3618abfb4' #@param {type:"string"}
#@markdown (cf. d079ec0e5b1a4c3d9b4301d3618abfb4, Geoapify에서 발급)

ylat, xlon = Geoapify_geocoding(location, report='y', apikey=api_key)
status_code1, respond_data1 = Geoapify_Isoline(lat=ylat[0], lon=xlon[0], cal_type ='distance', search_range=isolines.split())
map = spatial_visualizing(xcoord=xlon, ycoord=ylat, zoom= 15)
#중심점 추가
for num, latitude in enumerate(ylat):
  popup = folium.Popup('위도: {}<br>경도: {}'.format(round(xlon[num], 4), round(ylat[num], 4)), max_width=300)
  folium.Marker(location=[ylat[num], xlon[num]], popup=popup, tooltip=location,
                icon=folium.Icon(color='red', icon='info-sign')).add_to(map)
#등치선도 추가
df1, bins1 = df_and_bins_maker(resp_json=respond_data1, col_name='range')
choropleth = folium.Choropleth(geo_data=respond_data1, name="{}에서의 범위".format(location),
                               data=df1, columns=['id','range'], key_on="feature.properties.id",
                               fill_color="YlGn", fill_opacity=0.6, line_opacity=0.3,
                               legend_name="단위 (m)", bins=bins1, reset=True, smooth_factor=0, highlight=True).add_to(map)
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['range'], labels=False))
folium.LayerControl().add_to(map)
map

In [10]:
#지도 저장하기
map.save('isolines1.html')
from google.colab import files
files.download('/content/isolines1.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##특정지점의 영향 범위 표시하기2(Ctrl+Enter)<br>(등치선도, Isoline Map)

In [11]:
#@title 비교지역의 중심점 주소나 이름을 입력해주세요
location = '\uCDA9\uCCAD\uB300\uD559\uAD50' #@param {type:"string"}
#@markdown (cf. 한국교원대  or  충청북도 청주시 흥덕구 태성탑연로 250)
#@markdown ### 등치선도 범위를 입력해주세요
isolines = '2500' #@param {type:"string"}
#@markdown (cf.500 300 100  |  단, distance는 미터, time은 초단위로 입력해주세요)
#@markdown ### API Key를 입력해주세요
api_key = 'd079ec0e5b1a4c3d9b4301d3618abfb4' #@param {type:"string"}
#@markdown (cf. d079ec0e5b1a4c3d9b4301d3618abfb4, Geoapify에서 발급)

ylat, xlon = Geoapify_geocoding(location, report='y', apikey=api_key)
status_code2, respond_data2 = Geoapify_Isoline(lat=ylat[0], lon=xlon[0], cal_type ='distance', search_range=isolines.split())
map = spatial_visualizing(xcoord=xlon, ycoord=ylat, zoom= 15)
#중심점 추가
for num, latitude in enumerate(ylat):
  popup = folium.Popup('위도: {}<br>경도: {}'.format(round(xlon[num], 4), round(ylat[num], 4)), max_width=300)
  folium.Marker(location=[ylat[num], xlon[num]], popup=popup, tooltip=location,
                icon=folium.Icon(color='red', icon='info-sign')).add_to(map)
#등치선도 추가
df2, bins2 = df_and_bins_maker(resp_json=respond_data2, col_name='range')
choropleth =folium.Choropleth(geo_data=respond_data2, name="{}에서 {}까지의 범위".format(location, isolines),
                              data=df2, columns=['id','range'], key_on="feature.properties.id",
                              fill_color="YlGn", fill_opacity=0.6, line_opacity=0.3,
                              legend_name="단위 (m)", bins=bins2, reset=True, smooth_factor=0, highlight=True).add_to(map)
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['range'], labels=False))
folium.LayerControl().add_to(map)
map

In [12]:
#지도 저장하기
map.save('isolines2.html')
from google.colab import files
files.download('/content/isolines2.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##전체영향권 또는 교차영향권 찾기(Ctrl+Enter)<br>(등치선도 응용, Union & Intersection Map)

In [15]:
#@title 영향범위의 지오메트리 변환 방법을 입력해주세요
operation = "intersection" #@param ['intersection', 'union']
#@markdown (intersection =  교차영향권 | union = 전체영향권)
#@markdown ### API Key를 입력해주세요
api_key = 'ce8069542fd94ca2af8b17934f9510ce' #@param {type:"string"}
#@markdown (cf. ce8069542fd94ca2af8b17934f9510ce, Geoapify에서 Places API Key 발급)
statcode3, resp_data3 = Geoapify_Geometry(target_ids=[df1['id'][0], df2['id'][0]], operation=operation, apiKey=api_key)
map = spatial_visualizing(xcoord=xlon, ycoord=ylat, zoom= 15)
#지오메트리맵 추가
folium.Choropleth(geo_data=resp_data3, fill_opacity=0.7, line_opacity=0.2, reset=True, smooth_factor=0, highlight=True).add_to(map)
#레이어 컨트롤러 추가
folium.LayerControl().add_to(map)
map

In [ ]:
#지도 저장하기
map.save('union.html')
from google.colab import files
files.download('/content/union.html')